# Gradio Day!

Today we will build User Interfaces using the outrageously simple Gradio framework.

Prepare for joy!

Please note: your Gradio screens may appear in 'dark mode' or 'light mode' depending on your computer settings.

In [ ]:
!pip install -q transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.5 MB/s eta 0:00:00


In [ ]:
!pip install -q bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.2 MB/s eta 0:00:00


In [ ]:
!pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.4 MB/s eta 0:00:00


In [ ]:
!huggingface-cli login --token {"YOR_TOKEN"}


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `LLMs` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `LLMs`


In [ ]:
# Minimal imports for Hugging Face and Gradio
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import gradio as gr
from transformers import GenerationConfig

In [ ]:
DEVICE = "cpu"
if torch.backends.mps.is_available():
    DEVICE = "mps"
elif torch.cuda.is_available():
    DEVICE = "cuda"

print(f"Using device: {DEVICE}")

Using device: cuda


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "meta-llama/Llama-3.2-1B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto"  # or use torch.float16 if needed
)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
def generate_response(prompt, model, tokenizer, **generation_kwargs):
    """
    Generates a response from the model given a prompt string using Hugging Face transformers.
    Supports configurable generation settings like temperature, top_p, repetition_penalty, etc.
    """

    gen_config = getattr(model, "generation_config", GenerationConfig())

    default_generation_kwargs = {
        "do_sample": True,
        "top_p": 0.9,
        "temperature": 0.7,
        "max_new_tokens": 1024,
        "repetition_penalty": 1.2
    }

    generation_kwargs = {**default_generation_kwargs, **generation_kwargs}

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        **generation_kwargs
    )

    input_length = inputs["input_ids"].shape[1]

    return tokenizer.decode(outputs[0][input_length:], skip_special_tokens=True)


In [ ]:
def format_messages_as_prompt(messages, tokenizer):
    """
    Converts chat-style messages into a prompt using the tokenizer's chat template.
    """
    return tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

In [ ]:
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [ ]:
prompts = [
    {"role": "system", "content": "You are a helpeful assistant that responds in markdown"},
    {"role": "user", "content": user_prompt}
  ]

In [ ]:
prompt = format_messages_as_prompt(prompts, tokenizer)

response = generate_response(prompt, model, tokenizer)

print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A data-driven comedian walks into a room and says, "I've got a problem. I'm trying to predict the next big thing in machine learning, but my models keep making mistakes."

The other scientists stare at him, skeptical.

"I mean," he continues, "it's not just that they're wrong – it's more like... well, let me give you an example."

He pulls out his phone and starts scrolling through social media...

"...and see what happened with this one tweet:

"Machine learning model predicts stock price will go up by 20% today! #ML"

And then people start arguing about who bought all those stocks based on Twitter analysis..."

One scientist whispers, "Did we buy them?"

Another asks, "What did the algorithm say? 'Buy!' "

(The others groan.)

"That was a bit of a crash course!" 

How was that? Did I successfully made some numbers-crunching fun without losing any data points?

(Let me know if you want another attempt)


## User Interface time!

In [ ]:
# here's a simple function

def shout(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

In [ ]:
shout("hello")

Shout has been called with input hello


'HELLO'

In [ ]:
# The simplicty of gradio. This might appear in "light mode" - I'll show you how to make this in dark mode later.

gr.Interface(fn=shout, inputs="textbox", outputs="textbox").launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9139739b1c57bdc27c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
gr.Interface(fn=shout, inputs=[gr.Textbox(label="Input Text"), gr.Button("Convert")], outputs="textbox").launch()

/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1018: UserWarning: Expected 1 arguments for function <function shout at 0x7f18a76d2700>, received 2.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1026: UserWarning: Expected maximum 1 arguments for function <function shout at 0x7f18a76d2700>, received 2.
  warnings.warn(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://26a2562465bb6fea66.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# Adding share=True means that it can be accessed publically
# A more permanent hosting is available using a platform called Spaces from HuggingFace, which we will touch on next week
# NOTE: Some Anti-virus software and Corporate Firewalls might not like you using share=True. If you're at work on on a work network, I suggest skip this test.

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never", theme="dark").launch(share=True)

/usr/local/lib/python3.11/dist-packages/gradio/blocks.py:1116: UserWarning: Cannot load dark. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/dark (Request ID: Root=1-6819ec40-48a8c77268b02b164e638936;9237fa27-f187-4c5a-9efd-581740a9a2d0)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://93a28bfefa7fbdfb9f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr

def gradio_interface(prompt):
    prompts = [
        {"role": "system", "content": "You are a helpful assistant that responds in markdown"},
        {"role": "user", "content": prompt}
    ]
    formatted_prompt = format_messages_as_prompt(prompts, tokenizer)
    return generate_response(formatted_prompt, model, tokenizer)

view = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Textbox(label="Your message:", placeholder="Enter your message here...", lines=6),
    outputs=gr.Textbox(label="Response:", placeholder="Response from model will appear here...", lines=8),
    allow_flagging="never",
    theme="dark",
    title="LLaMA-3.2-1B Chatbot",
    description="Type a prompt and get a response from the LLaMA-3.2-1B-Instruct model."
)

view.launch(share=True)

/usr/local/lib/python3.11/dist-packages/gradio/blocks.py:1116: UserWarning: Cannot load dark. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/dark (Request ID: Root=1-681a1755-3c9a43d1280be291782201ca;7ec15659-02db-49ea-b738-bd9dddf62de7)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")
/usr/local/lib/python3.11/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f6339ad58f1e195e05.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Forcing dark mode

Gradio appears in light mode or dark mode depending on the settings of the browser and computer. There is a way to force gradio to appear in dark mode, but Gradio recommends against this as it should be a user preference (particularly for accessibility reasons). But if you wish to force dark mode for your screens, below is how to do it.

In [ ]:
# Define this variable and then pass js=force_dark_mode when creating the Interface

force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""
view = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Textbox(label="Your message:", placeholder="Enter your message here...", lines=6),
    outputs=gr.Textbox(label="Response:", placeholder="Response from model will appear here...", lines=8),
    js=force_dark_mode,  # اضافه کردن کد جاوااسکریپت به رابط
    allow_flagging="never",
    theme="dark",  # تم به‌صورت پیش‌فرض "dark"
    title="LLaMA-3.2-1B Chatbot",
    description="Type a prompt and get a response from the LLaMA-3.2-1B-Instruct model."
)

view.launch(share=True)

/usr/local/lib/python3.11/dist-packages/gradio/blocks.py:1116: UserWarning: Cannot load dark. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/dark (Request ID: Root=1-681a1758-5c9ad41b297d886c65374131;a91e1d07-f0b3-4a4f-897c-f2a9d70e02ea)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")
/usr/local/lib/python3.11/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b85280ce45da9633c0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# Inputs and Outputs

view = gr.Interface(
    fn=gradio_interface,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Markdown(label="Response:", lines=8)],
    flagging_mode="never"
)
view.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c3f63a3873c1231050.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
import time
from IPython.display import Markdown, display, update_display

def stream_llama_response_in_gradio(prompt, delay=0.01):
    """
    Streams the model output (simulated streaming) in Gradio using generate_response.
    """
    prompts = [
        {"role": "system", "content": "You are a helpful assistant that responds in markdown"},
        {"role": "user", "content": prompt}
    ]
    formatted_prompt = format_messages_as_prompt(prompts, tokenizer)

    full_response = generate_response(formatted_prompt, model, tokenizer)

    cleaned = full_response.replace("```", "").replace("markdown", "").strip()

    reply = ""
    for char in cleaned:
        reply += char
        yield reply
        time.sleep(delay)

In [ ]:
view = gr.Interface(
    fn=stream_llama_response_in_gradio,
    inputs=gr.Textbox(label="Your message:", placeholder="Enter your message here...", lines=6),
    outputs=gr.Markdown(label="Response:", placeholder="Response from model will appear here...", lines=8),
    js=force_dark_mode,
    allow_flagging="never",
    theme="dark",
    title="LLaMA-3.2-1B Chatbot",
    description="Type a prompt and get a response from the LLaMA-3.2-1B-Instruct model."
)

view.launch(share=True)

/usr/local/lib/python3.11/dist-packages/gradio/blocks.py:1116: UserWarning: Cannot load dark. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/dark (Request ID: Root=1-681a175e-524e018b512184d5107e06e7;345d4f04-926e-472e-8ed5-a6fd622432c9)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")
/usr/local/lib/python3.11/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bf6f41ee230ee4fcc5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Minor improvement

I've made a small improvement to this code.

Previously, it had these lines:

```
for chunk in result:
  yield chunk
```

There's actually a more elegant way to achieve this (which Python people might call more 'Pythonic'):

`yield from result`

I cover this in more detail in the Intermediate Python notebook in the week1 folder - take a look if you'd like more.

# Building a company brochure generator

Now you know how - it's simple!

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you read the next few cells</h2>
            <span style="color:#900;">
                Try to do this yourself - go back to the company brochure in week1, day5 and add a Gradio UI to the end. Then come and look at the solution.
            </span>
        </td>
    </tr>
</table>

In [ ]:
# A class to represent a Webpage
import requests
from bs4 import BeautifulSoup

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [31]:
system_message = (
    "You are an assistant that analyzes the contents of a company website landing page "
    "and creates a short brochure about the company for prospective customers, investors and recruits. "
    "Respond in markdown."
)

def stream_brochure(company_name, url):
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += Website(url).get_contents()

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]
    formatted_prompt = format_messages_as_prompt(messages, tokenizer)

    # گرفتن پاسخ کامل با تابع generate_response
    full_response = generate_response(formatted_prompt, model, tokenizer)

    # پاک‌سازی اختیاری markdownهای زائد
    cleaned = full_response.replace("```", "").replace("markdown", "").strip()

    # شبیه‌سازی استریم (کاراکتر به کاراکتر یا جمله‌ای)
    reply = ""
    for char in cleaned:
        reply += char
        yield reply
        time.sleep(0.005)  # شبیه‌سازی تأخیر

def stop_streaming(stop_signal):
    stop_signal["stop"] = True
    return gr.update()

In [30]:
with gr.Blocks(title="Company Brochure Generator", css="""
    .start-button, .stop-button {
        margin-top: 10px;
        width: 100%;
        text-align: center;
    }
""") as view:
    gr.Markdown("## 📄 Company Brochure Generator (LLaMA)")
    gr.Markdown("Generate a professional brochure from a company's website using the LLaMA model.")

    # ورودی‌ها
    company_input = gr.Textbox(label="🧾 Company Name", placeholder="e.g. OpenAI")
    url_input = gr.Textbox(label="🌐 Website URL (include http:// or https://)", placeholder="e.g. https://openai.com")

    # وضعیت توقف
    stop_signal = gr.State(value={"stop": False})

    # خروجی
    output_md = gr.Markdown(label="📄 Brochure")

    # دکمه‌ها
    with gr.Row():
        start_btn = gr.Button("🚀 Start", elem_classes="start-button")
        stop_btn = gr.Button("⛔ Stop", elem_classes="stop-button")

    # عملکرد دکمه‌ها
    start_btn.click(
        fn=lambda: {"stop": False},  # ریست توقف
        inputs=[],
        outputs=stop_signal
    ).then(
        fn=stream_brochure,
        inputs=[company_input, url_input, stop_signal],
        outputs=output_md
    )

    stop_btn.click(
        fn=stop_streaming,
        inputs=[stop_signal],
        outputs=[]
    )

view.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3baab7fa13009ac26a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
